In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from bs4 import BeautifulSoup
import requests
import dateutil.parser

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

def money_to_int(moneystring):
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

In [2]:
movie_source_21 = requests.get('https://www.boxofficemojo.com/daily/2021/?view=year')
movie_source_20 = requests.get('https://www.boxofficemojo.com/daily/2020/?view=year')
soup_21 = BeautifulSoup(movie_source_21.text, 'html5lib')
soup_20 = BeautifulSoup(movie_source_20.text, 'html5lib')

In [3]:
table_21 = soup_21.find('table')
table_20 = soup_20.find('table')
rows21 = [row for row in table_21.find_all('tr')]
rows20 = [row for row in table_20.find_all('tr')]

In [4]:
rows21[1]
links21 = []
for i in range(len(rows21)):
    if i != 0:
        link = rows21[i].find_all('td')[1].find('a')['href']
        links21.append(link)
links21
links20 = []
for i in range(len(rows20)):
    if i != 0:
        link = rows20[i].find_all('td')[1].find('a')['href']
        links20.append(link)
lonk = rows21[4].find_all('td')[1].find('a')['href']
links_total = links21 + links20
links_total

['/date/2021-09-26/?ref_=bo_di_table_1',
 '/date/2021-09-25/?ref_=bo_di_table_2',
 '/date/2021-09-24/?ref_=bo_di_table_3',
 '/date/2021-09-23/?ref_=bo_di_table_4',
 '/date/2021-09-22/?ref_=bo_di_table_5',
 '/date/2021-09-21/?ref_=bo_di_table_6',
 '/date/2021-09-20/?ref_=bo_di_table_7',
 '/date/2021-09-19/?ref_=bo_di_table_8',
 '/date/2021-09-18/?ref_=bo_di_table_9',
 '/date/2021-09-17/?ref_=bo_di_table_10',
 '/date/2021-09-16/?ref_=bo_di_table_11',
 '/date/2021-09-15/?ref_=bo_di_table_12',
 '/date/2021-09-14/?ref_=bo_di_table_13',
 '/date/2021-09-13/?ref_=bo_di_table_14',
 '/date/2021-09-12/?ref_=bo_di_table_15',
 '/date/2021-09-11/?ref_=bo_di_table_16',
 '/date/2021-09-10/?ref_=bo_di_table_17',
 '/date/2021-09-09/?ref_=bo_di_table_18',
 '/date/2021-09-08/?ref_=bo_di_table_19',
 '/date/2021-09-07/?ref_=bo_di_table_20',
 '/date/2021-09-06/?ref_=bo_di_table_21',
 '/date/2021-09-05/?ref_=bo_di_table_22',
 '/date/2021-09-04/?ref_=bo_di_table_23',
 '/date/2021-09-03/?ref_=bo_di_table_24',
 

In [5]:
def get_summary_dict(link_list):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - daily domestic gross
        - ranking and past ranking
        - percent gross change per day/week
        - day of gross
        - theaters released in and average gross per theater
        - distributer
    Return information as a dictionary.
    '''
    
    base_url = 'https://www.boxofficemojo.com'
    
   
    
    headers = ['Date', 'rank', 'rank_yest',
               'movie_title', 'daily_gross', 'gross_change_day%', 
               'gross_change_week%', 'num_of_theaters', 'avg_gross_per_theater', 
               'gross_to_date', 'days_in_release', 'distributor' ]
    ranks = []
    titles = []
    rank_yests = []
    daily_grosss = []
    gross_day_pcts = []
    gross_week_pcts = []
    theaterss = []
    avg_theaters = []
    gross_dates = []
    days_rels = []
    distribs = []
    dates = []
    for link in link_list:
        #Create full url to scrape
        url = base_url + link
    
        #Request HTML and parse
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page,"lxml")

    
    
        date = soup.find('h1').text.replace('Domestic Box Office For ', '')
    
        for tr in soup.find_all('tr')[1:]:
            tds = tr.find_all('td')
        
            titles.append(tds[2].text)
        
            ranks.append(tds[0].text)
        
            rank_yests.append(tds[1].text)
        
            raw_daily_gross = tds[3].text
            daily_grosss.append(raw_daily_gross)
        
            gross_day_pcts.append(tds[4].text)
        
            gross_week_pcts.append(tds[5].text)
        
            theaterss.append(tds[6].text)
        
            avg_theater = tds[7].text
            avg_theaters.append(avg_theater)
        
            gross_date_raw = tds[8].text
            gross_dates.append(gross_date_raw)
        
            days_rels.append(tds[9].text)

            distribs.append(tds[10].text.strip())
            dates.append(date)


  
                                    
                
    
    #Create movie dictionary and return
    movie_dict = dict(zip(headers, [dates, ranks, rank_yests, titles, daily_grosss, 
                                    gross_day_pcts, gross_week_pcts, theaterss, avg_theaters, 
                                    gross_dates, days_rels , distribs]))
    
    return movie_dict
                          


In [6]:
movies_dict = get_summary_dict(links_total)

In [7]:
movie_df = pd.DataFrame.from_dict(movies_dict)
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15015 entries, 0 to 15014
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Date                   15015 non-null  object
 1   rank                   15015 non-null  object
 2   rank_yest              15015 non-null  object
 3   movie_title            15015 non-null  object
 4   daily_gross            15015 non-null  object
 5   gross_change_day%      15015 non-null  object
 6   gross_change_week%     15015 non-null  object
 7   num_of_theaters        15015 non-null  object
 8   avg_gross_per_theater  15015 non-null  object
 9   gross_to_date          15015 non-null  object
 10  days_in_release        15015 non-null  object
 11  distributor            15015 non-null  object
dtypes: object(12)
memory usage: 1.4+ MB


In [8]:
movie_df['Date']= pd.to_datetime(movie_df['Date'])
movie_df.head()

,Date,rank,rank_yest,movie_title,daily_gross,gross_change_day%,gross_change_week%,num_of_theaters,avg_gross_per_theater,gross_to_date,days_in_release,distributor
0,2021-09-26,1,1,Shang-Chi and the Legend of the Ten Rings,"$3,530,763",-40.5%,-41.6%,"3,952",$893,"$196,205,495",24,Walt Disney Studios Motion Pictures
1,2021-09-26,2,2,Dear Evan Hansen,"$1,604,170",-37.3%,-,"3,364",$476,"$7,443,830",3,Universal Pictures
2,2021-09-26,3,3,Free Guy,"$1,079,530",-43.7%,-17.2%,"3,175",$340,"$114,121,092",45,20th Century Studios
3,2021-09-26,4,4,Candyman,"$634,720",-46%,-26.1%,"2,556",$248,"$56,890,355",31,Universal Pictures
4,2021-09-26,5,5,Jungle Cruise,"$457,420",-44.5%,-18.8%,"2,065",$221,"$114,893,077",59,Walt Disney Studios Motion Pictures


In [9]:
cdc_data = pd.read_csv('COVID-19_Vaccinations_in_the_United_States_Jurisdiction.csv')

In [10]:
cdc_data['Date'] = pd.to_datetime(cdc_data['Date'])
cdc_data.head()

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,...,Series_Complete_Pfizer_18Plus,Series_Complete_Unk_Manuf_18Plus,Series_Complete_Janssen_65Plus,Series_Complete_Moderna_65Plus,Series_Complete_Pfizer_65Plus,Series_Complete_Unk_Manuf_65Plus,Series_Complete_FedLTC,Series_Complete_FedLTC_Residents,Series_Complete_FedLTC_Staff,Series_Complete_FedLTC_Unknown
0,2021-09-22,38,PR,4938390,191100,2009880,2737410,0,154629,173381,...,1052663,48,22278,269897,251096,9,21221,3505,3522,14194
1,2021-09-22,38,ME,2110220,116300,892500,1101420,0,156986,178087,...,398071,208,22772,121310,121462,96,15478,7502,5656,2320
2,2021-09-22,38,UT,3955660,175400,1515860,2264400,0,123385,152595,...,741516,4,15471,131339,156098,1,21824,8047,7616,6161
3,2021-09-22,38,OH,14714495,687400,6038680,7988415,0,125882,147135,...,2837203,4275,85659,770432,838271,2189,149663,74156,48994,26513
4,2021-09-22,38,NH,2121780,120400,833000,1168380,0,156046,177026,...,394131,41,14191,94227,115338,21,21832,10710,8954,2168


In [16]:
movies_vaccinesdf = pd.merge(movie_df, cdc_data, on='Date')
movies_vaccinesdf.columns

Index(['Date', 'rank', 'rank_yest', 'movie_title', 'daily_gross',
       'gross_change_day%', 'gross_change_week%', 'num_of_theaters',
       'avg_gross_per_theater', 'gross_to_date', 'days_in_release',
       'distributor', 'MMWR_week', 'Location', 'Distributed',
       'Distributed_Janssen', 'Distributed_Moderna', 'Distributed_Pfizer',
       'Distributed_Unk_Manuf', 'Dist_Per_100K', 'Distributed_Per_100k_12Plus',
       'Distributed_Per_100k_18Plus', 'Distributed_Per_100k_65Plus',
       'Administered', 'Administered_12Plus', 'Administered_18Plus',
       'Administered_65Plus', 'Administered_Janssen', 'Administered_Moderna',
       'Administered_Pfizer', 'Administered_Unk_Manuf', 'Administered_Fed_LTC',
       'Administered_Fed_LTC_Residents', 'Administered_Fed_LTC_Staff',
       'Administered_Fed_LTC_Unk', 'Administered_Fed_LTC_Dose1',
       'Administered_Fed_LTC_Dose1_Residents',
       'Administered_Fed_LTC_Dose1_Staff', 'Administered_Fed_LTC_Dose1_Unk',
       'Admin_Per_100K',

In [24]:
movies_vaccinesdf.to_csv('Movies_and_CDC_Data.csv', header=['Date', 'rank', 'rank_yest', 'movie_title', 'daily_gross',
       'gross_change_day%', 'gross_change_week%', 'num_of_theaters',
       'avg_gross_per_theater', 'gross_to_date', 'days_in_release',
       'distributor', 'MMWR_week', 'Location', 'Distributed',
       'Distributed_Janssen', 'Distributed_Moderna', 'Distributed_Pfizer',
       'Distributed_Unk_Manuf', 'Dist_Per_100K', 'Distributed_Per_100k_12Plus',
       'Distributed_Per_100k_18Plus', 'Distributed_Per_100k_65Plus',
       'Administered', 'Administered_12Plus', 'Administered_18Plus',
       'Administered_65Plus', 'Administered_Janssen', 'Administered_Moderna',
       'Administered_Pfizer', 'Administered_Unk_Manuf', 'Administered_Fed_LTC',
       'Administered_Fed_LTC_Residents', 'Administered_Fed_LTC_Staff',
       'Administered_Fed_LTC_Unk', 'Administered_Fed_LTC_Dose1',
       'Administered_Fed_LTC_Dose1_Residents',
       'Administered_Fed_LTC_Dose1_Staff', 'Administered_Fed_LTC_Dose1_Unk',
       'Admin_Per_100K', 'Admin_Per_100k_12Plus', 'Admin_Per_100k_18Plus',
       'Admin_Per_100k_65Plus', 'Recip_Administered',
       'Administered_Dose1_Recip', 'Administered_Dose1_Pop_Pct',
       'Administered_Dose1_Recip_12Plus',
       'Administered_Dose1_Recip_12PlusPop_Pct',
       'Administered_Dose1_Recip_18Plus',
       'Administered_Dose1_Recip_18PlusPop_Pct',
       'Administered_Dose1_Recip_65Plus',
       'Administered_Dose1_Recip_65PlusPop_Pct', 'Series_Complete_Yes',
       'Series_Complete_Pop_Pct', 'Series_Complete_12Plus',
       'Series_Complete_12PlusPop_Pct', 'Series_Complete_18Plus',
       'Series_Complete_18PlusPop_Pct', 'Series_Complete_65Plus',
       'Series_Complete_65PlusPop_Pct', 'Series_Complete_Janssen',
       'Series_Complete_Moderna', 'Series_Complete_Pfizer',
       'Series_Complete_Unk_Manuf', 'Series_Complete_Janssen_12Plus',
       'Series_Complete_Moderna_12Plus', 'Series_Complete_Pfizer_12Plus',
       'Series_Complete_Unk_Manuf_12Plus', 'Series_Complete_Janssen_18Plus',
       'Series_Complete_Moderna_18Plus', 'Series_Complete_Pfizer_18Plus',
       'Series_Complete_Unk_Manuf_18Plus', 'Series_Complete_Janssen_65Plus',
       'Series_Complete_Moderna_65Plus', 'Series_Complete_Pfizer_65Plus',
       'Series_Complete_Unk_Manuf_65Plus', 'Series_Complete_FedLTC',
       'Series_Complete_FedLTC_Residents', 'Series_Complete_FedLTC_Staff',
       'Series_Complete_FedLTC_Unknown'], index=False)

In [1]:
#todo: Clean the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484115 entries, 0 to 484114
Data columns (total 80 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Date                                    484115 non-null  object 
 1   rank                                    484115 non-null  int64  
 2   rank_yest                               484115 non-null  object 
 3   movie_title                             484115 non-null  object 
 4   daily_gross                             484115 non-null  object 
 5   gross_change_day%                       484115 non-null  object 
 6   gross_change_week%                      484115 non-null  object 
 7   num_of_theaters                         484115 non-null  object 
 8   avg_gross_per_theater                   484115 non-null  object 
 9   gross_to_date                           484115 non-null  object 
 10  days_in_release                         4841